In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

In [2]:
fhvhv_202011 = pd.read_parquet('../fhvhv_tripdata_2020-11.parquet', engine='pyarrow')
fhvhv_202011.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B02884,B02884,2020-11-01 00:03:18,2020-11-01 00:06:29,2020-11-01 00:08:19,2020-11-01 00:16:49,157,82,1.98,...,0.90,0.00,NaN,0.0,6.62,N,N,,N,N
1,HV0003,B02884,B02884,2020-11-01 00:14:46,2020-11-01 00:19:57,2020-11-01 00:20:13,2020-11-01 00:30:35,82,7,3.86,...,1.29,0.00,NaN,0.0,12.64,N,N,,N,N
2,HV0003,B02884,B02884,2020-11-01 00:27:10,2020-11-01 00:36:01,2020-11-01 00:36:35,2020-11-01 00:56:51,7,36,7.57,...,2.56,0.00,NaN,0.0,24.70,N,N,,N,N
3,HV0003,B02884,B02884,2020-11-01 00:50:36,2020-11-01 00:59:17,2020-11-01 00:59:51,2020-11-01 01:04:41,36,36,0.90,...,0.93,0.00,NaN,0.0,13.56,N,N,,N,N
4,HV0003,B02682,B02682,2020-11-01 00:10:20,2020-11-01 00:16:13,2020-11-01 00:17:21,2020-11-01 00:30:23,249,232,2.07,...,2.26,2.75,NaN,9.5,25.81,N,N,,N,N


In [3]:
fhvhv_202011.shape

(11596865, 24)

In [4]:
fhvhv_202011.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11596865 entries, 0 to 11596864
Data columns (total 24 columns):
 #   Column                Non-Null Count     Dtype         
---  ------                --------------     -----         
 0   hvfhs_license_num     11596865 non-null  object        
 1   dispatching_base_num  11596865 non-null  object        
 2   originating_base_num  8367342 non-null   object        
 3   request_datetime      11596856 non-null  datetime64[ns]
 4   on_scene_datetime     8380962 non-null   datetime64[ns]
 5   pickup_datetime       11596865 non-null  datetime64[ns]
 6   dropoff_datetime      11596865 non-null  datetime64[ns]
 7   PULocationID          11596865 non-null  int64         
 8   DOLocationID          11596865 non-null  int64         
 9   trip_miles            11596865 non-null  float64       
 10  trip_time             11596865 non-null  int64         
 11  base_passenger_fare   11596865 non-null  float64       
 12  tolls                 1159

# 移除共乘紀錄

In [5]:
fhvhv_202011 = fhvhv_202011[fhvhv_202011.shared_match_flag == 'N']
fhvhv_202011.shape

(11595761, 24)

# 計算等待時間

司機到達載客地點的時間(on_scene_datetime)僅限無障礙車輛，所以有30%左右這一欄沒有資料  
是否可以透過其餘70%的資料推估所有等待時間? 若其他特徵都沒有明顯差異的情況下

In [6]:
cal_wating_time = fhvhv_202011[fhvhv_202011['on_scene_datetime'].notnull()]

In [7]:
cal_wating_time.shape

(8380961, 24)

排除包含不合理的時間的紀錄，駕駛抵達時間早於需求時間，乘客上車時間早於駕駛抵達時間

In [8]:
cal_wating_time = cal_wating_time[cal_wating_time['on_scene_datetime'] >= cal_wating_time['request_datetime']]

cal_wating_time = cal_wating_time[cal_wating_time['pickup_datetime'] >= cal_wating_time['on_scene_datetime']]

計算乘客等待時間(rider_waiting_time)、駕駛等待時間(driver_waiting_time)

In [9]:

cal_wating_time['rider_waiting_time'] = cal_wating_time['on_scene_datetime'] - cal_wating_time['request_datetime']
cal_wating_time['rider_waiting_time'] = round(cal_wating_time['rider_waiting_time'] / np.timedelta64(1, 's'), 0)


cal_wating_time['driver_waiting_time'] = cal_wating_time['pickup_datetime'] - cal_wating_time['on_scene_datetime']
cal_wating_time['driver_waiting_time'] = round(cal_wating_time['driver_waiting_time'] / np.timedelta64(1, 's'), 0)

cal_wating_time.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,rider_waiting_time,driver_waiting_time
0,HV0003,B02884,B02884,2020-11-01 00:03:18,2020-11-01 00:06:29,2020-11-01 00:08:19,2020-11-01 00:16:49,157,82,1.98,...,NaN,0.0,6.62,N,N,,N,N,191.0,110.0
1,HV0003,B02884,B02884,2020-11-01 00:14:46,2020-11-01 00:19:57,2020-11-01 00:20:13,2020-11-01 00:30:35,82,7,3.86,...,NaN,0.0,12.64,N,N,,N,N,311.0,16.0
2,HV0003,B02884,B02884,2020-11-01 00:27:10,2020-11-01 00:36:01,2020-11-01 00:36:35,2020-11-01 00:56:51,7,36,7.57,...,NaN,0.0,24.70,N,N,,N,N,531.0,34.0
3,HV0003,B02884,B02884,2020-11-01 00:50:36,2020-11-01 00:59:17,2020-11-01 00:59:51,2020-11-01 01:04:41,36,36,0.90,...,NaN,0.0,13.56,N,N,,N,N,521.0,34.0
4,HV0003,B02682,B02682,2020-11-01 00:10:20,2020-11-01 00:16:13,2020-11-01 00:17:21,2020-11-01 00:30:23,249,232,2.07,...,NaN,9.5,25.81,N,N,,N,N,353.0,68.0


In [10]:
cal_wating_time.shape

(8362311, 26)

In [11]:
cal_wating_time[['rider_waiting_time', 'driver_waiting_time']].describe()

,rider_waiting_time,driver_waiting_time
count,8.362311e+06,8.362311e+06
mean,1.833036e+02,6.055290e+01
std,1.495012e+02,4.873956e+01
min,0.000000e+00,0.000000e+00
25%,9.300000e+01,2.000000e+01
50%,1.490000e+02,5.000000e+01
75%,2.310000e+02,1.090000e+02
max,7.052000e+04,1.410400e+04


# 節日

聖誕節可能有連假，再確認

查看聯邦法定假日  
6月19日的六月節(Juneteenth National Independence Day)從2021年開始

In [12]:
from pandas.tseries.holiday import USFederalHolidayCalendar

cal = USFederalHolidayCalendar()
holidays = cal.rules

for holiday in holidays:
    print(holiday.name)

New Year's Day
Birthday of Martin Luther King, Jr.
Washington’s Birthday
Memorial Day
Juneteenth National Independence Day
Independence Day
Labor Day
Columbus Day
Veterans Day
Thanksgiving Day
Christmas Day


## 添加情人節、聖派翠克節、萬聖節

使用function HolidayCalendarFactory

In [15]:
from pandas.tseries.holiday import AbstractHolidayCalendar, USFederalHolidayCalendar, HolidayCalendarFactory, Holiday

holiday_to_add = [Holiday("Valentine's Day", month=2, day=14),
                  Holiday("St.Patrick's Day", month=3, day=17),
                  Holiday("Halloween", month=10, day=31)]

cus_cal = HolidayCalendarFactory('MyHoliday', base=USFederalHolidayCalendar, other=holiday_to_add, base_class=AbstractHolidayCalendar)

# for i in all_holiday.rules:
#     print(i)
holidays_2020 = cus_cal.holidays(cus_cal, start='2020-01-01', end='2020-12-31')
for date in holidays_2020:
    print(date)

2020-01-01 00:00:00
2020-01-20 00:00:00
2020-02-14 00:00:00
2020-02-17 00:00:00
2020-03-17 00:00:00
2020-05-25 00:00:00
2020-07-03 00:00:00
2020-09-07 00:00:00
2020-10-12 00:00:00
2020-10-31 00:00:00
2020-11-11 00:00:00
2020-11-26 00:00:00
2020-12-25 00:00:00


In [ ]:
holidays_2020 = holidays_2020.date
holidays_2020

array([datetime.date(2020, 1, 1), datetime.date(2020, 1, 20),
       datetime.date(2020, 2, 14), datetime.date(2020, 2, 17),
       datetime.date(2020, 3, 17), datetime.date(2020, 5, 25),
       datetime.date(2020, 7, 3), datetime.date(2020, 9, 7),
       datetime.date(2020, 10, 12), datetime.date(2020, 10, 31),
       datetime.date(2020, 11, 11), datetime.date(2020, 11, 26),
       datetime.date(2020, 12, 25)], dtype=object)

## 添加新column：is_holiday

In [ ]:
fhvhv_202011['is_holiday'] = np.where(fhvhv_202011.pickup_datetime.dt.date.isin(holidays_2020), True, False)
fhvhv_202011['is_holiday'].value_counts()

is_holiday
False    10836174
True       759587
Name: count, dtype: int64

## 檢查

In [ ]:
date_to_filter = pd.to_datetime('2020-11-11').date()
fhvhv_202011[fhvhv_202011.pickup_datetime.dt.date == date_to_filter]

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,is_holiday
4001358,HV0003,B02764,B02764,2020-11-11 00:11:55,2020-11-11 00:13:47,2020-11-11 00:15:47,2020-11-11 00:47:10,186,22,15.64,...,2.75,NaN,7.34,33.01,N,N,,N,N,True
4001359,HV0003,B02764,B02764,2020-11-11 00:01:00,2020-11-11 00:03:11,2020-11-11 00:04:07,2020-11-11 00:09:15,243,128,1.81,...,0.00,NaN,0.00,5.40,N,N,,N,N,True
4001360,HV0003,B02764,B02764,2020-11-11 00:06:58,2020-11-11 00:14:03,2020-11-11 00:15:38,2020-11-11 00:22:16,127,136,1.39,...,0.00,NaN,0.00,5.39,N,N,,N,N,True
4001361,HV0003,B02764,B02764,2020-11-11 00:22:05,2020-11-11 00:27:47,2020-11-11 00:28:32,2020-11-11 00:38:50,127,69,4.39,...,0.00,NaN,0.00,10.02,N,N,,N,N,True
4001362,HV0003,B02764,B02764,2020-11-11 00:35:07,2020-11-11 00:41:09,2020-11-11 00:41:45,2020-11-11 00:53:02,48,236,2.74,...,2.75,NaN,0.00,17.96,N,N,,N,N,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4376624,HV0003,B02512,B02512,2020-11-11 23:41:40,2020-11-11 23:44:08,2020-11-11 23:44:22,2020-11-11 23:53:58,162,211,3.05,...,2.75,NaN,0.00,8.19,N,N,,N,N,True
4376625,HV0003,B02512,B02512,2020-11-11 23:55:52,2020-11-11 23:56:40,2020-11-11 23:57:54,2020-11-12 00:05:56,211,13,1.63,...,2.75,NaN,0.00,5.84,N,N,,N,N,True
4376626,HV0003,B02765,B02765,2020-11-11 23:31:39,2020-11-11 23:38:07,2020-11-11 23:38:24,2020-11-12 00:37:34,138,265,50.98,...,0.00,NaN,0.00,147.27,N,N,,N,N,True
4376627,HV0003,B02764,B02764,2020-11-11 22:57:37,2020-11-11 22:58:41,2020-11-11 23:00:42,2020-11-11 23:25:49,163,212,11.87,...,2.75,NaN,0.00,25.71,N,N,,N,N,True


# 星期幾

# 相關性

In [ ]:
fhvhv_202011.corr()

ValueError: could not convert string to float: 'HV0003'